# A1 - Part 1: Dataset selection and preprocessing

In this notebook we prepare the dataset that will be used in the rest of the activity.

We will:

- Load the **Bike Sharing (hourly)** dataset (`hour.csv`).
- Explore the original variables and their meaning.
- Apply a log transform to the target (`cnt_log`).
- One-hot encode categorical variables and build an encoded dataset (`hour_encoded`).
- Prepare the final data for the models:
  - shuffle and sample 1500 patterns,
  - split into 80% train+validation and 20% test,
  - scale input features and the target variable.

The encoded dataframe and the preprocessed numpy arrays created here will be reused in:

- `A1_2_BP_From_Scratch_Experiments.ipynb`
- `A1_3_MLR_and_PyTorch_Comparison.ipynb`


In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler


In [2]:
hour = pd.read_csv('../data/hour.csv')

# Day dataframe Content

In [ ]:
hour.info()

In [ ]:
hour.describe()

# Hour dataframe content

In [ ]:
hour.head()

# Dataset format

- instant: unique record id
- dtday: date in YYYY-MM-DD format
- season: [1: spring, 2:summer, 3:fall, 4:winter]
- yr: [0: 2011, 1:2012]
- mnth: [1 to 12]
- hr: hour [0 to 23]
- weekday: day of the week [0..6] 
- holiday: [0: not holiday, 1:holiday]
- working day: [0: weeked or holiday 1: not weeked or holiday]
- weathersit :
  - Clear, Parly Cloudy
  - Mist, Cloudy
  - Light Snow, Light Rain, Thunderstorm 
  - Heavy Rain, Snow
- temp : Normalized temperature in Celsius (0.0 to 1.0). min=-8, max=+39 
- atemp: Normalized feeling temperature in Celsius (0.0 to 1.0). min=-16, max=+50
- hum: Normalized humidity (0.0 to 1.0) 
- windspeed: Normalized wind speed (0.0 to 1.0)
- casual: Casual users count
- registered: Registered users count
- cnt: Rental bikes count (casual + registered)

https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset

In [ ]:
# drop column(s) we don't need
hour = hour.drop(columns=['instant', 'dteday'], errors='ignore')

# check distinct values
print(f'Season: {hour["season"].unique()}')
print(f'Year: {hour["yr"].unique()}')
print(f'Month: {hour["mnth"].unique()}')
print(f'Hour: {hour["hr"].unique()}')
print(f'Weekday: {hour["weekday"].unique()}')
print(f'Holiday: {hour["holiday"].unique()}')
print(f'Working Day: {hour["workingday"].unique()}')
print(f'Weathersit: {hour["weathersit"].unique()}')

# Missing Values Analysis

In [ ]:
print(hour.isnull().any())

# Outliers
From the graph below we can see that there is a large number of bike rentals with low count, but they represent valid data.

In [ ]:
#sns.set(font_scale=1.0)
sns.histplot(hour["cnt"], kde=True, bins=100)

In [ ]:
# Cnt is higly skewed. Log transform should improve model's accuracy
hour['cnt_log'] = np.log1p(hour['cnt'])
sns.histplot(hour['cnt_log'], kde=True, bins=50)

# Note: we will try predicting cnt_log, and will convert back to cnt using np.expm1(y_pred_log)

# Ensure categorical values are represented correctly
We one-hot encode categorical values and save the result into a new dataset
The intent is to prepare the csv for usage later in the pipeline


In [ ]:
# One-hot encode categorical columns. 
categorical_cols = ['season', 'yr', 'mnth', 'hr', 'weekday', 'weathersit']

# categories have fixed values, known range
cat_map = {
    "season": [1, 2, 3, 4],      
    "yr": [0, 1],                       
    "mnth": list(range(1,13)),                              
    "hr": list(range(24)),                               
    "weekday": list(range(0,7)),      
    "weathersit": list(range(1,5))                       
}

# Format categories for OneHotEncoder (ie: list of category lists)
categories_for_encoder = [cat_map[c] for c in categorical_cols]

# Build encoder with explicit categories
# I don't set ignore error on purpose, as I want to surface them
encoder = OneHotEncoder(
    categories=categories_for_encoder,
    sparse_output=False
)

# Validate config (from given categories)
encoded = encoder.fit_transform(hour[categorical_cols])
encoded_cols = encoder.get_feature_names_out(categorical_cols)
encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=hour.index)

# Assemble encoded dataset
hour_encoded = pd.concat([hour.drop(columns=categorical_cols), encoded_df], axis=1)

# Save the encoded dataset
hour_encoded.to_csv("../data/hours_encoded.csv", index=False) 

hour_encoded.head()

## Part 1.2 – Preparing the preprocessed dataset for the models

Now that we have the encoded dataset `hour_encoded`, we prepare the data in the exact
format that will be used by all the models (MLR, manual BP, PyTorch):

- select the regression target `cnt_log`,
- drop the original `cnt` column from the input features,
- randomly sample 1500 patterns,
- split into 80% train+validation and 20% test,
- apply feature scaling to inputs and target.

The resulting NumPy arrays will be used directly in the following notebooks of the assignment.


In [ ]:

# Start from the encoded dataset
# Copy created in order to not modify the original reference
data = hour_encoded.copy()

print("Encoded dataset shape:", data.shape)
data.head()


In [ ]:

# Defining target and feature matrix

target_col = "cnt_log"

# We can safely drop the original 'cnt' if it is still present, because we are going to predict cnt_log instead.
if "cnt" in data.columns:
    data = data.drop(columns=["cnt"])

# Separate features (X) and target (y)
X_df = data.drop(columns=[target_col])
y = data[target_col].values

print("Feature matrix shape:", X_df.shape)
print("Target vector shape :", y.shape)
print("Number of features  :", X_df.shape[1])


In [ ]:

# Shuffle and sample 1500 patterns. We randomly sample 1500 rows to match the experiments

data_sampled = data.sample(n=1500, random_state=42).reset_index(drop=True)

X_df = data_sampled.drop(columns=[target_col])
y = data_sampled[target_col].values

print("After sampling 1500 rows:")
print("Feature matrix shape:", X_df.shape)
print("Target vector shape :", y.shape)


# Train / Test split (80% / 20%)

X_trainval_df, X_test_df, y_trainval, y_test = train_test_split(
    X_df, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print("\nTrain+Val size:", X_trainval_df.shape[0])
print("Test size     :", X_test_df.shape[0])

# Convert to NumPy arrays
X_trainval = X_trainval_df.values
X_test = X_test_df.values

print("\nX_trainval shape:", X_trainval.shape)
print("X_test shape     :", X_test.shape)
print("y_trainval shape :", y_trainval.shape)
print("y_test shape     :", y_test.shape)


In [ ]:

# Feature scaling (StandardScaler for X). We scale the input features so that all models (MLR, manual BP, PyTorch) use the same normalized data.

x_scaler = StandardScaler()
X_trainval_np = x_scaler.fit_transform(X_trainval)
X_test_np     = x_scaler.transform(X_test)


# Target scaling (StandardScaler for y). It's also needed to scale the target value

y_scaler = StandardScaler()
y_trainval_scaled = y_scaler.fit_transform(y_trainval.reshape(-1, 1)).ravel()
y_test_scaled     = y_scaler.transform(y_test.reshape(-1, 1)).ravel()

n_features = X_trainval_np.shape[1]
print("Number of input features:", n_features)


## Summary of preprocessing

In this notebook we combined two main steps:

1. **Data understanding and encoding** (first part, by my teammate):
   - Loaded the original Bike Sharing hourly dataset (`hour.csv`).
   - Removed unnecessary columns.
   - Inspected categorical variables and their value ranges.
   - Checked for missing values.
   - Observed that `cnt` is highly skewed and created a log-transformed target `cnt_log`.
   - One-hot encoded categorical variables into a new dataframe `hour_encoded`.
   - Saved the encoded dataset into `hours_encoded.csv`.

2. **Preparing the final dataset for the models** (second part):
   - Started from `hour_encoded`.
   - Selected `cnt_log` as the regression target and dropped `cnt`.
   - Randomly sampled 1500 patterns for the experiments.
   - Split the data into:
     - 80% for training + validation (`X_trainval`, `y_trainval`),
     - 20% for test (`X_test`, `y_test`).
   - Applied **StandardScaler** to:
     - the input features (`X_trainval_np`, `X_test_np`),
     - the target values (`y_trainval_scaled`, `y_test_scaled`).

The following variables are now ready to be used in the next notebooks:

- `X_trainval_np`, `X_test_np`  → scaled input features
- `y_trainval`, `y_test`        → original target values
- `y_trainval_scaled`, `y_test_scaled` → scaled target values
- `x_scaler`, `y_scaler`        → fitted scalers (for inverse transformation)
- `n_features`                  → number of input features

These will be reused in:
- **A1_2_BP_From_Scratch_Experiments.ipynb** (manual backprop network)
- **A1_3_MLR_and_PyTorch_Comparison.ipynb** (MLR and PyTorch models).
